In [10]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch
import models

# 初始化模型和优化器
audio_model = models.Uni_CMAEFT(img_size=160, audio_length=1024, label_dim=10,  encoder_depth=12)
optimizer = torch.optim.Adam(audio_model.parameters(), lr=0.0001)

# 生成随机输入数据
audio = torch.randn(1, 1024, 128)
video = torch.randn(1, 3, 16, 160, 160)

# 前向传播
loss = audio_model(audio, video, mode='multimodal')

loss = loss.mean()
# 反向传播
loss.backward()

print(audio_model)

# # 检查梯度是否计算正确
# for name, param in audio_model.named_parameters():
#     if param.grad is None:
#         print(f"参数 {name} 没有梯度")
#     else:
#         pass
#         #print(f"参数 {name} 的梯度正常")

# # 保存参数的初始值
# initial_params = {name: param.clone() for name, param in audio_model.named_parameters()}

# # 更新参数
# optimizer.step()

# # 检查参数是否更新
# for name, param in audio_model.named_parameters():
#     if not torch.equal(param, initial_params[name]):
#         pass
#         #print(f"参数 {name} 已更新")
#     else:
#         print(f"参数 {name} 未更新")


img_size (160, 160) patch_size (16, 16) frames 16 t_patch_size 2
Number of Audio Patches: 512, Visual Patches: 800
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 8, 100, 768])
Uni_CMAEFT(
  (patch_embed_a): Tokenizer_audio(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (patch_embed_v): Tokenizer_video(
    (proj): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
  )
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (norm1_a): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (norm1_v): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )

In [18]:
import torch
import models

# 初始化模型和优化器
audio_model = models.Uni_CMAE(img_size=160, audio_length=1024,  norm_pix_loss=True, encoder_depth=12,pred_t_dim=16)
optimizer = torch.optim.Adam(audio_model.parameters(), lr=0.0001)

# 生成随机输入数据
audio = torch.randn(1, 1024, 128)
video = torch.randn(1, 3, 16, 160, 160)

# 前向传播
loss, loss_mae, loss_mae_a, loss_mae_v, loss_c, mask_a, mask_v, c_acc = audio_model(audio, video,)

loss = loss.mean()
# 反向传播
loss.backward()



# 检查梯度是否计算正确
for name, param in audio_model.named_parameters():
    if param.grad is None:
        print(f"参数 {name} 没有梯度")
    else:
        pass
        #print(f"参数 {name} 的梯度正常")

# 保存参数的初始值
initial_params = {name: param.clone() for name, param in audio_model.named_parameters()}

# 更新参数
optimizer.step()

# 检查参数是否更新
for name, param in audio_model.named_parameters():
    if not torch.equal(param, initial_params[name]):
        pass
        #print(f"参数 {name} 已更新")
    else:
        print(f"参数 {name} 未更新")


A Uni-CMAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
img_size (160, 160) patch_size (16, 16) frames 16 t_patch_size 2
Number of Audio Patches: 512, Visual Patches: 800
pos_embed_v_t shape: torch.Size([1, 8, 768])
pos_embed_v_s shape: torch.Size([1, 100, 768])
pos_embed_v shape: torch.Size([1, 8, 100, 768])
decoder_pos_embed_v_t shape: torch.Size([1, 8, 512])
decoder_pos_embed_v_s shape: torch.Size([1, 100, 512])
decoder_pos_embed_v shape: torch.Size([1, 8, 100, 512])
decoder_pos_embed_v_trans shape: torch.Size([1, 800, 512])
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 8, 100, 768])
参数 pos_embed_a 没有梯度
参数 pos_embed_v_t 没有梯度
参数 pos_embed_v_s 没有梯度
参数 pos_embed_v 没有梯度
参数 decoder_pos_embed_a 没有梯度
参数 decoder_pos_embed_v_t 没有梯度
参数 decoder_pos_embed_v_s 没有梯度
参数 decoder_pos_embed_v 没有梯度
参数 decoder_pos_embed_v_trans 没有梯度
参数 blocks.0.norm1.weight 没有梯度
参数 blocks.0.norm1.bias 没有梯度
参数 blocks.0.norm2.weight 

In [14]:
import torch
import models
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import dataloader as dataloader

pth_path = '/home/hao/Project/uni-cmae/egs/voxceleb2/exp/testmae02-audioset-uni-cmae-balNone-lr5e-5-epoch6-bs40-normTrue-c0.01-p1.0-tpFalse-mr-0.9-0.75/models/best_audio_model.pth'

# 初始化模型和优化器
audio_model = models.Uni_CMAE(img_size=160, audio_length=1024, norm_pix_loss=True, encoder_depth=12, tr_pos=False)

mdl_weight = torch.load(pth_path, map_location=torch.device('cpu'))

# 处理键名不匹配问题
new_state_dict = {}
for k, v in mdl_weight.items():
    if k.startswith('module.'):
        k = k[len('module.'):]
    new_state_dict[k] = v

# 调整 pos_embed_v 和 decoder_pos_embed_v 的形状
if 'pos_embed_v' in new_state_dict:
    new_state_dict['pos_embed_v'] = new_state_dict['pos_embed_v'].view(audio_model.pos_embed_v.shape)
if 'decoder_pos_embed_v' in new_state_dict:
    new_state_dict['decoder_pos_embed_v'] = new_state_dict['decoder_pos_embed_v'].view(audio_model.decoder_pos_embed_v.shape)

msg = audio_model.load_state_dict(new_state_dict, strict=False)

print('missing_keys:', msg.missing_keys)
print('unexpected_keys:', msg.unexpected_keys)

A Uni-CMAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
img_size (160, 160) patch_size (16, 16) frames 16 t_patch_size 2
Number of Audio Patches: 512, Visual Patches: 800
pos_embed_v_t shape: torch.Size([1, 8, 768])
pos_embed_v_s shape: torch.Size([1, 100, 768])
pos_embed_v shape: torch.Size([1, 8, 100, 768])
decoder_pos_embed_v_t shape: torch.Size([1, 8, 512])
decoder_pos_embed_v_s shape: torch.Size([1, 100, 512])
decoder_pos_embed_v shape: torch.Size([1, 8, 100, 512])
decoder_pos_embed_v_trans shape: torch.Size([1, 800, 512])
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 8, 100, 768])
missing_keys: []
unexpected_keys: []


In [37]:
import torch
import models
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import dataloader as dataloader


val_audio_conf = {
    'num_mel_bins': 128,
    'target_length': 1024,
    'freqm': 0,
    'timem': 0,
    'mixup': 0,
    'dataset': 'audioset',
    'mode': 'pretrain', # val & finetuen & test & pretrain
    'noise': False,
    'im_res': 160,
    'mean': -5.081,  # 添加 norm_mean
    'std': 4.4849    # 添加 norm_std
}
video_conf = {
    'dataset_type' : 'video', 
    'aa_type': "rand-m7-n4-mstd0.5-inc1",
    'pretrain_rand_flip': True,
    'pretrain_rand_erase_prob': 0.25,
    'pretrain_rand_erase_mode': "pixel",
    'pretrain_rand_erase_count': 1,
    'pretrain_rand_erase_split': False,
    'jitter_aspect_relative': [0.75, 1.3333],
    'jitter_scales_relative': [0.5, 1.0],
    'repeat_aug': 1,
    'num_retries': 10,
    'train_jitter_scales': (256, 320),
    'train_crop_size': 160,
    'train_random_horizontal_flip': True,
    'test_num_ensemble_views': 10,
    'test_num_spatial_crops': 3,
    'test_crop_size': 160,
    'sampling_rate': 4,
    'num_frames': 16,
    'target_fps': 30,
    'mean': (0.45, 0.45, 0.45),
    'std': (0.225, 0.225, 0.225),
    'enable_multi_thread_decode': False,
    'inverse_uniform_sampling': False,
    'use_offset_sampling': True
}

val_set = dataloader.AudiosetDataset(
    dataset_json_file='/home/hao/Project/uni-cmae/egs/voxceleb2/test_data.json',
    label_csv='/home/hao/Project/uni-cmae/egs/voxceleb2/class_labels_indices.csv',
    audio_conf=val_audio_conf,
    video_conf=video_conf,
)

# 获取 val_set 中的一个样本
audio, video, label = val_set[18]

print(f'audio shape: {audio.shape}, video shape: {video.shape}, label: {label}')


Dataset has 36237 samples
Using Label Smoothing: 0.0
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
using video data as video input
use dataset mean -5.081 and std 4.485 to normalize the input.
not use noise augmentation
number of classes is 1
now in pretrain mode.
now use frame -1 from total 10 frames
now using 160 * 160 image input
jitter_aspect_relative [0.75, 1.3333] jitter_scales_relative [0.5, 1.0]
Constructing DATASET pretrain...
Perform standard augmentation
crop_size=(256, 320, 160)
audio shape: torch.Size([1024, 128]), video shape: torch.Size([3, 16, 160, 160]), label: tensor([1.])
